# Data exploration with the Bivariate Gaussian

In this notebook, I explore winery data with the two-dimensional Gaussian by varying the covariance matrix, drawing random samples from the resulting distribution, and plotting contour lines of the density

In [1]:
# Standard includes
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Useful module for dealing with the Gaussian density
from scipy.stats import norm, multivariate_normal 
# installing packages for interactive graphs
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider

In [2]:
# Load data set.
data = np.loadtxt('wine.data.txt', delimiter=',')
# Names of features
featurenames = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash','Magnesium', 'Total phenols', 
                'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 
                'OD280/OD315 of diluted wines', 'Proline']
# Split 178 instances into training set (trainx, trainy) of size 130 and test set (testx, testy) of size 48
np.random.seed(0)
perm = np.random.permutation(178)
trainx = data[perm[0:130],1:14]
trainy = data[perm[0:130],0]
testx = data[perm[130:178], 1:14]
testy = data[perm[130:178],0]

In [3]:
# Fits a gaussian to features
def fit_gaussian(x,features):
    mu = np.mean(x[:,features], axis=0)
    covar=np.cov(x[:,features], rowvar=0, bias=1)
    return mu, covar

In [4]:
f1=0
f2=6
label=1
mu, covar = fit_gaussian(trainx[trainy==label], [f1,f2])
print("Mean:\n" + str(mu))
print("Covariance matrix:\n" + str(covar))

Mean:
[13.78534884  2.99627907]
Covariance matrix:
[[0.23325279 0.07526874]
 [0.07526874 0.15240941]]


In [5]:
# Find range of data
def find_range(x):
    lower = min(x)
    upper = max(x)
    width = upper - lower
    lower = lower - 0.2 * width
    upper = upper + 0.2 * width
    return lower, upper

In [6]:
# Plot density contours for 2 given classes
def plot_contours(mu,covar,x1g,x2g,col):
    rv = multivariate_normal(mean=mu,cov=covar)
    z=np.zeros((len(x1g),len(x2g)))
    for i in range(0,len(x1g)):
        for j in range(0,len(x2g)):
            z[j,i]=rv.logpdf([x1g[i], x2g[j]])
    sign, logdet = np.linalg.slogdet(covar)
    normalizer = -0.5*(2*np.log(6.28)+sign*logdet)
    for offset in range(1,4):
        plt.contour(x1g,x2g,z, levels=[normalizer - offset], colors=col, linewidths=2.0, linestyles='solid') 

In [7]:
# Plot density contours for 2 given classes. Also shows correlation of two features
@interact_manual( f1=IntSlider(0,0,12,1), f2=IntSlider(6,0,12,1), label=IntSlider(1,1,3,1) )
def two_features_plot(f1,f2,label):
    if f1 == f2:
        print("Please choose different features for f1 and f2.")
        return 
    
    x1_lower, x1_upper =find_range(trainx[trainy==label,f1])
    x2_lower, x2_upper =find_range(trainx[trainy==label,f2])
    plt.xlim(x1_lower, x1_upper)
    plt.ylim(x2_lower, x2_upper)
    
    res=200
    x1g=np.linspace(x1_lower,x1_upper)
    x2g=np.linspace(x2_lower,x2_upper)
    
    mu, covar = fit_gaussian(trainx[trainy==label], [f1,f2])
    plot_contours(mu,covar,x1g,x2g,'k')
    
    plt.xlabel(featurenames[f1], fontsize=14, color='red')
    plt.ylabel(featurenames[f2], fontsize=14, color='red')
    plt.title('Class ' + str(label), fontsize=14, color='blue')
    plt.show()

interactive(children=(IntSlider(value=0, description='f1', max=12), IntSlider(value=6, description='f2', max=1…

In [8]:
# Plot density contours for all 3 classes. Also shows correlation of two features
@interact_manual( f1=IntSlider(0,0,12,1), f2=IntSlider(6,0,12,1) )
def three_class_plot(f1,f2):
    if f1 == f2:
        print("Please choose different features for f1 and f2.")
        return
    
    x1_lower, x1_upper =find_range(trainx[:,f1])
    x2_lower, x2_upper =find_range(trainx[:,f2])
    plt.xlim(x1_lower, x1_upper)
    plt.ylim(x2_lower, x2_upper)
    
    
    colors = ['r', 'k', 'g']
    for label in range(1,4):
        plt.plot(trainx[trainy==label,f1], trainx[trainy==label,f2], marker='o', ls='None', c=colors[label-1])
    
    
    res=200
    x1g=np.linspace(x1_lower,x1_upper)
    x2g=np.linspace(x2_lower,x2_upper)
    
    means=[]
    covars=[]
    
    mu1, covar1 = fit_gaussian(trainx[trainy==1],[f1,f2])
    mu2, covar2 = fit_gaussian(trainx[trainy==2],[f1,f2])
    mu3, covar3 = fit_gaussian(trainx[trainy==3],[f1,f2])
    
    plot_contours(mu1,covar1,x1g,x2g,colors[0])    
    plot_contours(mu2,covar2,x1g,x2g,colors[1])
    plot_contours(mu3,covar3,x1g,x2g,colors[2])
    
    plt.xlabel(featurenames[f1], fontsize=14, color='red')
    plt.ylabel(featurenames[f2], fontsize=14, color='red')
    plt.title('Wine data', fontsize=14, color='blue')
    plt.show()
    

interactive(children=(IntSlider(value=0, description='f1', max=12), IntSlider(value=6, description='f2', max=1…

In [9]:
pi=np.zeros((4,1))
leny=len(trainy)
for label in range(1,4):
    pi[label] = len(trainy[trainy==label])/len(trainy)

## Building Bivariate Generative Model

In [10]:
# Fits bivariate generative model based on two selected features
def fit_generative_model(x, y, features):
    k = 3 # number of classes
    d = len(features) # number of features
    mu = np.zeros((k+1,d)) # list of means
    covar = np.zeros((k+1,d,d)) # list of covariance matrices
    pi = np.zeros(k+1) # list of class weights
    for label in range(1,k+1):
        indices = (y==label)
        mu[label,:], covar[label,:,:] = fit_gaussian(x[indices,:], features)
        pi[label] = float(sum(indices))/float(len(y))
    return mu, covar, pi

## Model Testing

In [11]:
# Tests bivariate generative model and computes error of model
@interact(f1=IntSlider(0,0,12,1), f2=IntSlider(6,0,12,1))

def test_model(f1,f2):
    if f1 == f2: # need f1 != f2
        print("Please choose different features for f1 and f2.")
        return
    
    features=[f1,f2]

    mu, covar, pi=fit_generative_model(trainx,trainy,[f1,f2])
    
    k=3
    leny=len(testx)
    score=np.zeros((leny,k+1))
    
    for i in range(0,leny):
        for label in range(1,k+1):
            score[i,label]=np.log(pi[label]) + \
            multivariate_normal.logpdf(testx[i, features], mean=mu[label,:], cov=covar[label,:,:])
    predictions=np.argmax(score[:,1:4], axis=1)+1
        
    errors = np.sum(predictions!=testy)
    print("Test error using feature(s): "),
    for f in features:
        print("'" + featurenames[f] + "'" + " "),
    print
    print("Errors: " + str(errors) + "/" + str(leny))
    
    percent_error=errors/leny*100
    print(percent_error)
        

interactive(children=(IntSlider(value=0, description='f1', max=12), IntSlider(value=6, description='f2', max=1…

## Explore Decision Boundaries 

In [12]:
# Shows bivariate generative model and decision boundaries
@interact(f1=IntSlider(0,0,12,1), f2=IntSlider(6,0,12,1))
def show_decision_boundary(f1,f2):
    
    mu, covar, pi = fit_generative_model(trainx, trainy, [f1,f2])
    
    x1_lower, x1_upper = find_range(trainx[:,f1])
    x2_lower, x2_upper = find_range(trainx[:,f2])
    plt.xlim([x1_lower,x1_upper])
    plt.ylim([x2_lower,x2_upper])

    colors = ['r', 'k', 'g']
    for label in range(1,4):
        plt.plot(trainx[trainy==label,f1], trainx[trainy==label,f2], marker='o', ls='None', c=colors[label-1])

    res = 200
    x1g = np.linspace(x1_lower, x1_upper, res)
    x2g = np.linspace(x2_lower, x2_upper, res)
    
    random_vars = {}
    for label in range(1,4):
        random_vars[label] = multivariate_normal(mean=mu[label,:],cov=covar[label,:,:])

    Z = np.zeros((len(x1g), len(x2g)))
    for i in range(0,len(x1g)):
        for j in range(0,len(x2g)):
            scores = []
            for label in range(1,4):
                scores.append(np.log(pi[label]) + random_vars[label].logpdf([x1g[i],x2g[j]]))
            Z[i,j] = np.argmax(scores) + 1
            
    plt.contour(x1g,x2g,Z.T,3,cmap='seismic')
    
    plt.xlabel(featurenames[f1], fontsize=14, color='red')
    plt.ylabel(featurenames[f2], fontsize=14, color='red')
    plt.show()

interactive(children=(IntSlider(value=0, description='f1', max=12), IntSlider(value=6, description='f2', max=1…